## Notes
忽略Keras RL的部分。Stable-baselines在下面  
这个现在用的是gym 0.26.2, 在monitor.py里有一个return的问题可能需要更改return的value数量才能兼容  
PPO感觉挺好的，在只120步的时候可以超过random选择的return。DQN在训练120步的时候不行。

## Keras RL DQN

In [1]:
!pip install -e disneyenv


Obtaining file:///gpfs/data/oermannlab/users/hz2212/DS-GA%203001-007/disneyenv
  Preparing metadata (setup.py) ... done
  Running setup.py develop for disneyenv


In [2]:
import gymnasium as gym
import disneyenv

env = gym.make('disneyenv/Disney-v0') 


/gpfs/data/oermannlab/users/hz2212/DS-GA 3001-007/disneyenv/disneyenv/envs/disney.py:42: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  self.waittime = pd.read_csv(


AssertionError: action space does not inherit from `gymnasium.spaces.Space`, actual type: <class 'gym.spaces.discrete.Discrete'>

In [ ]:
import numpy as np
import pandas as pd

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Input,Dropout,Reshape
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [4]:
def build_model(input_shape, output_shape):
    model = Sequential([
        Dense(512,activation="relu",input_shape = (1,input_shape)),
        Dropout(0.2),
        Flatten(),
        Dense(256,activation = "relu"),
        Dropout(0.2),
        Dense(128,activation = "relu"),
        Dense(64,activation = "relu"),
        Dense(output_shape,activation = "linear"),
        Flatten()
    ])
    model.summary()
    return model

In [5]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=2000,window_length = 1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=10)
    return dqn

In [6]:
model = build_model(env.observation_space.n,env.action_space.n)
dqn_agent = build_agent(model,env.action_space.n)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 512)            118784    
                                                                 
 dropout (Dropout)           (None, 1, 512)            0         
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8

In [7]:
dqn_agent.compile(Adam(learning_rate=0.001))

In [8]:
dqn_agent.fit(env, nb_steps = 1000, visualize = False, verbose = 1) 

Training for 1000 steps ...


c:\Users\82751\anaconda3\envs\my_environment\lib\site-packages\gymnasium\utils\passive_env_checker.py:174: UserWarning: WARN: Future gymnasium versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\82751\anaconda3\envs\my_environment\lib\site-packages\gymnasium\utils\passive_env_checker.py:187: UserWarning: WARN: Future gymnasium versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(


A new day! Today is 2018-7-26
Interval 1 (0 steps performed)


c:\Users\82751\anaconda3\envs\my_environment\lib\site-packages\gymnasium\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
c:\Users\82751\anaconda3\envs\my_environment\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 1:21:38 - reward: 1.0000

c:\Users\82751\anaconda3\envs\my_environment\lib\site-packages\gymnasium\utils\passive_env_checker.py:133: UserWarning: WARN: The obs returned by the `step()` method should be an int or np.int64, actual type: <class 'numpy.ndarray'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
c:\Users\82751\anaconda3\envs\my_environment\lib\site-packages\gymnasium\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


   11/10000 [..............................] - ETA: 46:40 - reward: -35.2727

c:\Users\82751\anaconda3\envs\my_environment\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 1000/10000 [==>...........................] - ETA: 49:28 - reward: -29.2526done, took 330.622 seconds


## Stable Baseline3 DQN 

In [1]:
!pip install -e disneyenv

Obtaining file:///gpfs/data/oermannlab/users/hz2212/DS-GA%203001-007/disneyenv
  Preparing metadata (setup.py) ... done
  Attempting uninstall: disneyenv
    Found existing installation: disneyenv 0.0.1
    Uninstalling disneyenv-0.0.1:
      Successfully uninstalled disneyenv-0.0.1
  Running setup.py develop for disneyenv


In [2]:
import gym
import disneyenv

env = gym.make("disneyenv/Disney-v0")

/gpfs/data/oermannlab/users/hz2212/DS-GA 3001-007/disneyenv/disneyenv/envs/disney.py:42: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  self.waittime = pd.read_csv(


In [ ]:
# from stable_baselines3.common.env_checker import check_env
# check_env(env)

In [3]:
import torch
from stable_baselines3 import PPO, DQN

device = "cuda" if torch.cuda.is_available() else "cpu"
agent = PPO("MlpPolicy",env,verbose=1, device=device)
agent.learn(total_timesteps=120)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

 A new day! Today is 2018-04-23

 A new day! Today is 2017-07-22

 A new day! Today is 2017-07-02

 A new day! Today is 2018-01-07

 A new day! Today is 2017-11-02

 A new day! Today is 2018-09-11

 A new day! Today is 2017-10-17

 A new day! Today is 2018-07-02

 A new day! Today is 2017-08-14

 A new day! Today is 2018-08-13

 A new day! Today is 2018-07-12

 A new day! Today is 2018-05-14

 A new day! Today is 2018-06-01

 A new day! Today is 2017-06-16

 A new day! Today is 2017-10-26

 A new day! Today is 2017-07-22

 A new day! Today is 2018-03-28

 A new day! Today is 2018-08-13

 A new day! Today is 2018-05-11

 A new day! Today is 2018-05-06

 A new day! Today is 2017-06-07

 A new day! Today is 2017-08-08

 A new day! Today is 2017-09-10

 A new day! Today is 2017-06-01

 A new day! Today is 2017-06-16

 A new day! Today is 2017-10-04

 A new day! Today is 2017-10-06

 A new day! T

In [5]:
obs = env.reset()
return_val = []
while True:
    action, _states = agent.predict(obs, deterministic=True)
    #action = env.action_space.sample()
    obs, reward, done,info = env.step(action)
    print("at",env.current_time,"agent go to ride",action,"and get reward",reward)
    return_val += [reward]

    if done:
        print(return_val)
        break



 A new day! Today is 2018-07-12
at 2018-07-12 08:35:08.425107 agent go to ride 15 and get reward 20.0
at 2018-07-12 08:57:08.425107 agent go to ride 15 and get reward 10.0
at 2018-07-12 09:22:26.710539 agent go to ride 90 and get reward 20.0
at 2018-07-12 09:42:26.710539 agent go to ride 90 and get reward 10.0
at 2018-07-12 10:09:44.995971 agent go to ride 15 and get reward 5.0
at 2018-07-12 10:31:44.995971 agent go to ride 15 and get reward 2.5
at 2018-07-12 10:53:44.995971 agent go to ride 15 and get reward 1.25
at 2018-07-12 11:29:03.281403 agent go to ride 90 and get reward 5.0
at 2018-07-12 12:04:03.281403 agent go to ride 90 and get reward 2.5
at 2018-07-12 12:31:21.566835 agent go to ride 15 and get reward 0.625
at 2018-07-12 12:53:21.566835 agent go to ride 15 and get reward 0.3125
at 2018-07-12 13:43:39.852267 agent go to ride 90 and get reward 1.25
at 2018-07-12 14:28:39.852267 agent go to ride 90 and get reward 0.625
at 2018-07-12 15:13:39.852267 agent go to ride 90 and get